## LAB 3
### 절단값 추정

#### 모형 적합.. (LAB 2 참조)

#### 로지스틱 모형을 적합 - logit_cc

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

buy=pd.read_table('buytest.txt', delimiter=" ")

buy_com = buy.dropna()
buy_com = buy_com.drop(["ID"], axis = 1)

buy_com = pd.get_dummies(buy_com, drop_first=True,                            
                         columns= ['SEX', 'LOC', 'ORGSRC'])

buydata = buy_com.drop(["C1","C2","C3","C4","C5","C6","C7","PURCHTOT","CLIMATE"]
                       , axis = 1)

train, test = train_test_split(buydata, test_size = 0.3, random_state = 1)
print('train dim: ',train.shape,'\n test dim:', test.shape)
y_train = train.loc[:,"RESPOND"]
X_train = train.drop("RESPOND", axis = 1)

y_test = test.loc[:,"RESPOND"]
X_test = test.drop("RESPOND", axis = 1)

X_train["intercept"] = 1.0 # for intercept
X_test["intercept"] = 1.0

logit = sm.Logit(y_train,X_train)
logit_res = logit.fit(maxiter = 1000)

train dim:  (6454, 27) 
 test dim: (2767, 27)
Optimization terminated successfully.
         Current function value: 0.253546
         Iterations 7


#### 절단값 설정하기

In [2]:
best_acc = 0
pred = np.array(logit_res.predict(X_test))

for i in range(100):
    pred_label = np.array( [pred[j] > 0.01 * i for j in range(pred.shape[0])] )
    acc = np.mean( pred_label == y_test )
    if acc > best_acc :
        best_acc = acc
        cut = 0.01 * i
        best_pred = pred_label
        
print('best acc : %2.2f, cut-off : %.2f' %(100 * best_acc, cut))

best acc : 92.48, cut-off : 0.42


In [3]:
for i in range(20):
    pred_label = np.array( [pred[j] > 0.05 * i for j in range(pred.shape[0])] )
    acc = np.mean( pred_label == y_test )
    print('cut-off : %.2f \t Accuracy : %.4f' %(i * 0.05, acc))

cut-off : 0.00 	 Accuracy : 0.0755
cut-off : 0.05 	 Accuracy : 0.3773
cut-off : 0.10 	 Accuracy : 0.7915
cut-off : 0.15 	 Accuracy : 0.8847
cut-off : 0.20 	 Accuracy : 0.9082
cut-off : 0.25 	 Accuracy : 0.9187
cut-off : 0.30 	 Accuracy : 0.9212
cut-off : 0.35 	 Accuracy : 0.9245
cut-off : 0.40 	 Accuracy : 0.9245
cut-off : 0.45 	 Accuracy : 0.9248
cut-off : 0.50 	 Accuracy : 0.9245
cut-off : 0.55 	 Accuracy : 0.9245
cut-off : 0.60 	 Accuracy : 0.9245
cut-off : 0.65 	 Accuracy : 0.9245
cut-off : 0.70 	 Accuracy : 0.9245
cut-off : 0.75 	 Accuracy : 0.9245
cut-off : 0.80 	 Accuracy : 0.9245
cut-off : 0.85 	 Accuracy : 0.9245
cut-off : 0.90 	 Accuracy : 0.9245
cut-off : 0.95 	 Accuracy : 0.9245


## 벌점화 회귀 분석
### 자료 설명
- Toyota의 중고차 판매에 대한 자료
- 51개의 변수와 1436개의 관측값으로 구성 : 분석에는 10개의 변수만 사용
- 변수 설명
    - Price : 판매 가격
    - Age_08_04 : 자료 취득 시점까지 자동차 구동 시간
    - KM : 주행거리
    - Fuel_Type : 연료유형 (CNG, Diesel, Patrol)
    - HP : 마력
    - Met_Color : 금속 색이면 1, 아니면 0
    - Automatic : 자동변속이면 1, 아니면 0
    - cc : 실린더 부피
    - Quartely_Tax : 분기별 세금 (\$)
    - Weight : 무게(kg)
    
### 분석 목적 : 나머지 변수들로 Price를 예측하기
    
### 데이터 불러오기

In [4]:
import pandas as pd
import numpy as np

toyota = pd.read_csv('ToyotaCorolla.csv', delimiter = ',')
toyota.head()


,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,?TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 분석에 사용할 변수만 선택

In [5]:
toyota = toyota.iloc[:,[2, 3, 6, 7, 8, 9, 11, 12, 16, 17]]
toyota.columns

Index(['Price', 'Age_08_04', 'KM', 'Fuel_Type', 'HP', 'Met_Color', 'Automatic',
       'cc', 'Quarterly_Tax', 'Weight'],
      dtype='object')

### 자료 전처리 1: 결측치 수 확인 및 제거

In [6]:
toyota.isnull().sum()

Price            0
Age_08_04        0
KM               0
Fuel_Type        0
HP               0
Met_Color        0
Automatic        0
cc               0
Quarterly_Tax    0
Weight           0
dtype: int64

### 자료 전처리 2: 범주형 변수 가변수화 (Fuel_type)

In [7]:
toyota = pd.get_dummies(toyota, drop_first = True)
print(toyota.shape)
toyota.head(4)

(1436, 11)


,Price,Age_08_04,KM,HP,Met_Color,Automatic,cc,Quarterly_Tax,Weight,Fuel_Type_Diesel,Fuel_Type_Petrol
0,13500,23,46986,90,1,0,2000,210,1165,1,0
1,13750,23,72937,90,1,0,2000,210,1165,1,0
2,13950,24,41711,90,1,0,2000,210,1165,1,0
3,14950,26,48000,90,0,0,2000,210,1165,1,0


### 자료 전처리 3 : 학습자료 / 시험자료 분리

In [8]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
train, test = train_test_split(toyota, test_size = 0.2, random_state = 42)
X_train, y_train = train.drop("Price", axis = 1), train.Price
X_test, y_test = test.drop("Price", axis = 1), test.Price

### 능형 회귀 분석 (Ridge)

#### 교차 확인을 통해 초모수 $\lambda$ 추정

In [9]:
import sklearn.linear_model as lm

alphas = (np.arange(1000)+1) * 0.01
cv_ridge = lm.RidgeCV(alphas,fit_intercept = True, normalize = True) # cross validation
cv_ridge.fit(X_train, y_train)
cv_ridge.alpha_

0.1

#### 교차 확인을 통해 추정한 $\lambda$를 이용해서 계수 추정

In [10]:
ridge = lm.Ridge(alpha = cv_ridge.alpha_, fit_intercept = True, normalize = True)
ridge.fit(X_train, y_train)
ridge.coef_

array([-1.07851070e+02, -1.92724298e-02,  2.02647643e+01,  7.75290795e+01,
        1.39328383e+02, -7.63780849e-03,  8.46930980e+00,  2.12199440e+01,
       -2.68242901e+02,  1.22553526e+03])

### 라쏘 회귀 분석 (LASSO)

#### 교차 확인을 통해 초모수 $\lambda$ 추정

In [11]:
cv_lasso = lm.LassoCV(n_alphas = 100, fit_intercept = True, normalize = True)
cv_lasso.fit(X_train, y_train)
cv_lasso.alpha_

2.86413567408479

#### 교차 확인을 통해 추정한 $\lambda$를 이용해서 계수 추정

In [12]:
lasso = lm.Lasso(alpha = cv_lasso.alpha_, fit_intercept = True, normalize = True)
lasso.fit(X_train, y_train)
lasso.coef_

array([-1.19579510e+02, -1.70628045e-02,  2.00403424e+01,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  2.00031837e+01,
       -0.00000000e+00,  4.78772202e+02])

#### 단순 회귀 모형과의 계수 크기 비교

In [13]:
slm = lm.LinearRegression(fit_intercept = True, normalize = True)
slm.fit(X_train, y_train)
slm.coef_ 

array([-1.19775791e+02, -1.57976065e-02,  1.43190481e+01,  4.80522278e+01,
        1.14554381e+02, -4.01635372e-02,  1.22428301e+01,  2.37010228e+01,
        5.53649605e+01,  2.60519859e+03])

In [14]:
l2 = np.mean(lasso.coef_[1:]**2)**0.5
print('L2 norm of Ridge coefficient : %3.0f' %l2)

l2 = np.mean(slm.coef_[1:]**2)**0.5
print('L2 norm of simple linear coefficient : %3.0f' %l2)

l1 = np.mean(abs(lasso.coef_[1:]))
print('L1 norm of LASSO coefficient : %3.0f' %l1)

l1 = np.mean(abs(slm.coef_[1:]))
print('L1 norm of simple linear coefficient : %3.0f' %l1)

L2 norm of Ridge coefficient : 160
L2 norm of simple linear coefficient : 870
L1 norm of LASSO coefficient :  58
L1 norm of simple linear coefficient : 319


#### 예측력 비교

In [15]:
err = np.mean((ridge.predict(X_train)-y_train)**2)
print('Train error of Ridge regression : %.0f' %err)

err = np.mean((lasso.predict(X_train)-y_train)**2)
print('Train error of LASSO regression : %.0f' %err)

err = np.mean((slm.predict(X_train)-y_train)**2)
print('Train error of simple regression : %.0f' %err)

print('\n\n\n')

err = np.mean((ridge.predict(X_test)-y_test)**2)
print('Prediction error of Ridge regression : %.0f' %err)

err = np.mean((lasso.predict(X_test)-y_test)**2)
print('Prediction error of LASSO regression : %.0f' %err)

err = np.mean((slm.predict(X_test)-y_test)**2)
print('Prediction error of simple regression : %.0f' %err)

Train error of Ridge regression : 1717099
Train error of LASSO regression : 1764563
Train error of simple regression : 1646820




Prediction error of Ridge regression : 2081208
Prediction error of LASSO regression : 2049526
Prediction error of simple regression : 2166693


### Elastic Net

- 초모수를 2개 추정해야하므로 계싼량이 급증
- Validation set을 이용

In [16]:
train1, validate = train_test_split(toyota, test_size = 0.2, random_state = 42)
X_train1, y_train1 = train1.drop("Price", axis = 1), train1.Price
X_val, y_val = validate.drop("Price", axis = 1), validate.Price

ratio = np.arange(1,10) * 0.1
best_acc = 1e+10
accs = []

for rate in ratio :
    cv_elastic_net = lm.ElasticNetCV(l1_ratio = rate, n_alphas = 100, fit_intercept = True, normalize = True)
    cv_elastic_net.fit(X_train1, y_train1)
    
    elastic_net = lm.ElasticNet(alpha = cv_elastic_net.alpha_, l1_ratio = rate, fit_intercept = True, normalize = True)
    elastic_net.fit(X_train1, y_train1)
    
    acc = np.mean((elastic_net.predict(X_val) - y_val)**2 )
    accs.append(acc)
    if acc < best_acc:
        best_rate = rate
        best_alpha = cv_elastic_net.alpha_
        best_acc = acc

elastic_net = lm.ElasticNet(alpha = best_alpha, l1_ratio = best_rate, 
                            fit_intercept = True, normalize = True, max_iter = 5e+4)
elastic_net.fit(X_train, y_train)


err = np.mean((elastic_net.predict(X_test)-y_test)**2)
print('Prediction error of Elastic Net : %.0f' %err)

elastic_net.coef_

Prediction error of Elastic Net : 10468896


array([-1.23368071e+01, -3.94738644e-03,  5.29865680e+00,  5.20640150e+01,
        3.50536786e+01,  6.08866509e-02,  1.23839025e+00,  2.93479484e+00,
        2.70251219e+01, -9.05832335e+00])